In [ ]:
%%capture
!pip install -q \
  numpy==1.26.4 \
  scipy==1.11.4 \
  scikit-learn==1.3.2 \
  pandas==2.1.4 \
  matplotlib==3.8.2 \
  opencv-python==4.8.1.78 \
  imageio


!pip install -q torch accelerate==0.33.0
!pip install imageio-ffmpeg


!pip install -q git+https://github.com/huggingface/diffusers.git

!pip install -q \
  transformers==4.44.2 \
  sentencepiece



!git clone https://github.com/google-deepmind/physics-IQ-benchmark.git

!pip install -q --no-deps -r physics-IQ-benchmark/requirements.txt


In [ ]:
import torch
from diffusers import CogVideoXImageToVideoPipeline,DiffusionPipeline
from diffusers.utils import export_to_video
from PIL import Image
from tqdm import tqdm
import pandas as pd
import glob
import os
import gc

In [ ]:
os.chdir("/teamspace/studios/this_studio/physics-IQ-benchmark")
!python code/download_physics_iq_data.py
os.chdir("/teamspace/studios/this_studio")

In [ ]:
gc.collect()
torch.cuda.empty_cache()

model_id = "THUDM/CogVideoX-5b-I2V"
print("Loading model... this may take a few minutes.")
pipe = CogVideoXImageToVideoPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16
).to("cuda")

pipe.enable_model_cpu_offload()
pipe.enable_sequential_cpu_offload()
pipe.vae.enable_tiling()

print("Model loaded successfully!")

In [ ]:
import torch

def count_params(module):
    return sum(p.numel() for p in module.parameters())

def count_trainable_params(module):
    return sum(p.numel() for p in module.parameters() if p.requires_grad)

total = 0
trainable = 0

for name, component in pipe.components.items():
    if hasattr(component, "parameters"):
        p = count_params(component)
        tp = count_trainable_params(component)
        total += p
        trainable += tp
        print(f"{name:15s} | params: {p/1e6:.2f}M | trainable: {tp/1e6:.2f}M")

print("-" * 60)
print(f"TOTAL params     : {total/1e6:.2f}M")
print(f"Trainable params : {trainable/1e6:.2f}M")


In [ ]:
df = pd.read_csv('/teamspace/studios/this_studio/physics-IQ-benchmark/descriptions/descriptions.csv')
df.columns

In [ ]:
df.info()

In [ ]:
switch_frames_dir = "/teamspace/studios/this_studio/physics-IQ-benchmark/physics-IQ-benchmark/switch-frames"
output_dir = "/teamspace/studios/this_studio/evaluated_videos"
os.makedirs(output_dir, exist_ok=True)

switch_frames = sorted(glob.glob(os.path.join(switch_frames_dir, "*.jpg")))
print(f"Found {len(switch_frames)} test cases.")


system_prompt = """Generate a physically accurate video continuation that follows real-world physics laws. 
Ensure proper object motion, gravity effects, collision dynamics, and momentum conservation. 
The following description provides the physical scenario and expected behavior: """

i = 0
for frame_path in tqdm(switch_frames):
    filename = os.path.basename(frame_path)
    
    video_id = filename.split("_")[0]
    
    output_filename = filename.replace("switch-frames_anyFPS", "generated").replace(".jpg", ".mp4")
    output_path = os.path.join(output_dir, output_filename)
    
    if os.path.exists(output_path):
        continue
    
    image = Image.open(frame_path).convert("RGB")
    
    desc = df['description'][i]
    prompt = system_prompt + " " + desc
    
    frames = pipe(
        prompt=prompt,
        image=image,                      
        num_videos_per_prompt=1,
        num_inference_steps=60,                        
        num_frames=49,    
        guidance_scale=6.0,                  
        generator=torch.Generator("cuda").manual_seed(42),              
    ).frames[0]
    
    export_to_video(frames, output_path, fps=8)
    i += 1

print(f"Generation complete. Videos saved to {output_dir}")

In [ ]:
# change_fps

In [ ]:
generated_videos_dir = "/teamspace/studios/this_studio/evaluated_videos"
output_score_dir = "/teamspace/studios/this_studio/scores"
descriptions_path = "/teamspace/studios/this_studio/physics-IQ-benchmark/descriptions/descriptions.csv"

!python3 /teamspace/studios/this_studio/physics-IQ-benchmark/code/run_physics_iq.py \
    --input_folders {generated_videos_dir} \
    --output_folder {output_score_dir} \
    --descriptions_file {descriptions_path}

import glob
csv_files = glob.glob(os.path.join(output_score_dir, "*.csv"))
for f in csv_files:
    print(f"Results in: {f}")
    with open(f, 'r') as file:
         print(file.read())